In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


# Execution Control 

**Learning Outcomes**

To understand:

* Signaling Errors 
* The conditional `if`
* Atomic conditional evaluation 
* Vector conditional evaluation
* Protected evaluation
* Other useful control statements

# Introduction

Execution control is used when we want to evaluate expressions only if a certain criterion is met. In this section, we will focus on how to apply execution control and when to use it. 

In [9]:
stringMe:{[input] if[input = 0; :"False"] "True"}
stringMe 0
stringMe 1

False
True


# Signaling Errors 
Before discussing any further, it's important to know how to return a custom error from within a function. 

In [ ]:
{[] a:10; b: `this; a+b }[] //this will throw a type error 

If we want to raise our own custom error message during execution we can use `'`, known as [signal](https://code.kx.com/q/ref/signal/):


In [11]:
//{[]a:10;b:`this; '"Out of cereal - send help!"; a+b }[]  //this error is thrown before we reach the type error 

Statements after an error is encountered in a function or script are not executed. 

# The Conditional `if`

The conditional `if` first performs a check to see whether a given condition is true or false - depending on which, it will take the following actions:
* if the statement is true: execute all subsequent statements
* if the statement is false: do not evaluate the subsequent statements and continue to the next q expression.

Let's start with a simple example - instead of providing a conditional statement to be evaluated, we can just use `1b` to have our `if` evaluate the subsequent statements

In [4]:
a:10    // initialize two sample variables
b:20
if[1b;a:a+1]    // condition is true, execute all subsequent statements
if[0b;b:b+1]
a   //incremented by if
b   //not changed as the condition wasn't met 

11
20


The `if` function can take many expressions to be evaluated in the event that the condition is met: 

In [14]:
x:10;
if[.z.t < 12:00:00 ;1"Evaluating within the if ... "; 
        x:x+1; //incrementing x 
        y:10; //defining new variable y 
        -1"Complete";
        y]    //note there is no value returned from an if!

x //checking if x is incrementing 

11
Evaluating within the if ... Complete


There is one way in which you can return from within an `if` statement and that is to use the force return `:` that we have encountered previously: 

In [15]:
x:10;
if[.z.t > 12:00:00 ;1"Evaluating within the if ... "; 
        :.z.t; //return the time - subsequent statements not evaluated 
        x+:1; //incrementing x 
        y:10; //defining new variable y 
        -1"Complete";
        y]
x

10


Whoops! Looks like that didn't work as we expected! That's right - force return only "makes sense" within a function, so wrapping this into a function: 


In [19]:
x:10;
{[]if[.z.t > 12:00:00 ;1"Evaluating within the if ... "; 
        :.z.t; //return the time - subsequent statements not evaluated 
        x+:1; //incrementing x 
        y:10; //defining new variable y 
        -1"Complete";
        y] "HI THERE"}[]
x  // not updated as other expression not evaluated after the force return

HI THERE
10


##### Typical usage 
In terms of where `if` is used, it is often included at the beginning of functions and scripts to check that the input passed is of the correct format. 

If the format of a variable is not right, we might do any of the following: 
* Write to stderr and exit the process early (within script execution)
* Write to stderr and/or throw an error within a function
* Write to stdout and assign the value to a pre-determined default

Here's an example: 

In [ ]:
var1:`a
if[not -19h=type var1;
    -2 "Error: Variable var1 input not of type: 19h";  //writing to stderr
    /exit 1]   //within a script, we might exit early after throwing the error
    /'"Inappropriate value passed for var1 - expected input time, got type:", string type var1;  //custom error
    -1 "Variable var1 not of the right type, defaulting to the current time:", string t:.z.t;
    var1:t]
var1

 ##### Why exit early? 
In the below example, we perform mathematical operations on our input: 

In [20]:
f:{[num] n:num*200f; 
        n:n%20;
        n:xexp[n;2];  //xexp can be an expensive operation 
        n:n*til count n;
        (),n}

f2:{[num] n:num*200f; 
        if[all n=0; :(),n];  //exiting execution early if our input is all zero
        n:n%20;  0N! n; 
        n:xexp[n;2];
        n:n*til count n;
        (),n}

\ts:2000 f[1000#0]   //without early return
\ts:2000 f2[1000#0]  //with early return
f2[1000#0]~f[1000#0] //output the same, even with early exit - important to ensure output types are consistent

50 41296
9 24912
1b


##### Exercise
Define a variable `r` to be 100 and `ans` to be an empty string. Write an `if` statement to say if `r` is greater than 85, then add 10 to `r`, and change `ans` to say "high".

In [ ]:
show r:100  //setting variable
show ans:""
if[r>85;r+:10;ans:"high"]
r
ans

In [27]:
// Write your code here
r:100
ans:" "
if[r>85; r:r+10; ans:"high"]
r
ans


110
high


# Atomic Conditional Evaluation
For conditional evaluation in kdb+/q,  we use different keywords depending on whether we are dealing with an atomic or vector input - we'll see that the atomic version can serve as either a simple 'if-else' (as can the vector) but also it can be extended to deal with multiple else conditions. 

## If-else

An [if-else](https://code.kx.com/q/ref/cond/) is similar to `if`, but instead of only executing when the condition is true, an if-else allows us to specify expressions to be executed when the condition is false (`0b`) also. Unlike `if`, it will return a value.

Below we will look at a simple example of how to convert a string to capital letters if the condition is true.

In [34]:
$[1b;upper string `hey;string `hey] //if[condition;ifTrueStatement;ifFalseStatement]
$[0b;upper "hey";"hey"]

HEY
hey


We can generalise the above example by creating a function that relies on a global context setting (the value of `CAPS_ON`) and returns a string of the passed datatype, either capitalized or not, depending on the setting.

In [35]:
CAPS_ON:0b                                  // declare a global "setting"
toString:{ if[not 10h=type x; x:string x];  // first check the type 
            $[CAPS_ON;upper x;x]}           // note the boolean value itself is enough, we don't need a statement
toString`hey

hey


In [36]:
CAPS_ON:1b                                        // change setting
toString`hey

HEY


We can use any condition that evaluates to a boolean in our conditional statement:

In [37]:
f:{[val] r:$[val within 20 30 ;   //specifying the range between 20-30 so we can check if a value is inside/outside 
    "within";
    "outside"];
        r," range"}
f[50]
f[25]

outside range
within range


If we have multiple expressions we want to evaluate, we can do so by wrapping the expression in square brackets - `[]`.  

In [5]:
balance: 19.32  //bank account balance 
beFrugal: $[balance < 10 ;  //if balance getting low 
            [2 "Under 10.00 in bank - transfer money stat!"];
            [1 "Go Shopping!!!"]]
beFrugal  //value has been assigned by final expression outputs

1
Go Shopping!!!

#### What changes if we have a balance of `19.32`? What value does `beFrugal` have in that case? 

In [ ]:
balance: 19.32  //changing bank account balance 
beFrugal: $[balance < 10 ;  //reasigning beFrugal 
            [2 "Under 10.00 in bank - transfer money stat!";1b];
            [1 "Go Shopping!!!";0b;]]
beFrugal  //Doesn't return 0b because of semicolon - LET'S GO SHOPPING

##### Comparison tests can be applied to any datatype

Any type that is a whole number (e.g. int, short, long), byte, or is a whole number underneath (e.g. time, timestamp) can be used in boolean checks. 

In [7]:
show time:0p
$[time;`after_midnight;`midnight]

midnight
2000.01.01D00:00:00.000000000


In [8]:
show time:time+1
$[time;`after_midnight;`midnight]

after_midnight
2000.01.01D00:00:00.000000001


##### Exercise

Create a function `chocPrices` that takes a symbol as an input. If the symbol is `mars` It should return the price for a Mars chocolate bar which is 2.5. 

If you give the function any other symbol, then it prints "Why bother?!" to standard error and returns 0. 

In [15]:
chocPrices:{$[x~`mars;2.5;[-2 "Why bother?!"; 0]]}
chocPrices `mars

2.5


In [16]:
chocPrices `twix

0


Why bother?!


In [24]:
//write your function here
chocPrices:{[input]
            $[input = `mars;2.5; [-2"Why bother";0]]}
chocPrices `mar


0


Why bother


## Extended form of if-else

We can extend our if-else to evaluate multiple statements if we have many checks we need to make - there is an extended usage of `$`.

Let's imagine we are capturing data from a sensor on a machine that deals with a woolen textile - Wool has an effective ironing range of 160–170 °C. A temperature lower than this might be ineffective and merit a warning, but a temperature above this would melt the fabric and be catastrophic.

Let's encode appropriate responses into a `tempMonitor` function: 

In [25]:
thresholds: 160 170 // temperature 
tempMonitor:{[sensorTemp;thresholds]   
                $[sensorTemp> thresholds[1]; 
                    '"Temperature too hot! Stop Immediately!!";      //we want to signal an error 
                    sensorTemp < thresholds[0]; 
                    -2"Temperature too low - ironing ineffective";   //we want to warn, so print to stderr
                    -1"Temperature within range"]; //this is optional - we don't have to include a final else 
                 sensorTemp}

In [26]:
tempMonitor[162;thresholds]   //this should be within our temperature range

162
Temperature within range


In [27]:
tempMonitor[150;thresholds]    //this should be below our temperature range

150


Temperature too low - ironing ineffective


In [28]:
tempMonitor[172;thresholds]    //this should be above our temperature range

QError: Temperature too hot! Stop Immediately!!

We can also return functions from our conditionals and use the functions -  assuming that the input required for each function is consistent across conditions i.e. they all take the same input and number of inputs. 

 ##### Group discussion

The below example builds on our previous example and combines a number of topics: 
* Indexing
* User defined functions
* Projections
* Signaling errors 
* Signaling to stderr

In [29]:
//temperature Actions
tooLow:{-2"Temperature too low - ironing ineffective"; min (0,x)}    //what should happen when too low
tooHigh:{'"Temperature too hot! Stop Immediately!!"};                //what should happen when too high
justRight:(::)   //function null - do nothing when fine

//temperature Thresholds 
woolThresh: 160 170      //for wool 
viscoseThresh: 150 180   //for viscose 

//core monitor logic
tempMonitor:{[sensorTemp;thresh]   
                function:$[sensorTemp> thresh[1]; 
                        tooHigh;
                    sensorTemp < thresh[0]; 
                        tooLow;
                        justRight]; //since we need a function to call we do need to include here
                 function[sensorTemp]}

//projections for each material
woolMonitor:tempMonitor[;woolThresh];
viscoseMonitor:tempMonitor[;viscoseThresh];

In [30]:
//checking for temperature values 
woolMonitor[162]

162


In [31]:
//checking for temperature values 
woolMonitor[150]

0


Temperature too low - ironing ineffective


In [32]:
//checking for temperature values 
viscoseMonitor[182]

QError: Temperature too hot! Stop Immediately!!

Consider the above example: 
* What assumptions are made about the inputs to our functions? 
* What input would cause outright errors (e.g. a type error)? 
* What input would cause logical errors (errors that aren't going to cause our code to throw an error, but would mean our logical requirements aren't being met)? 
* How could you improve the function to avoid these errors? 
* What other issues can you think of? 

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> This isn't exhaustive, and also in some cases might not be applicable. There's a big difference between writing code for user facing applications, versus "back-end" code - as with most things, the more you know about what you're trying to achieve before you start, the better!</i></p>
 

##### Exercise 

Create a dyadic function that compares the type of the first argument to a symbol, if the first argument is a symbol print the second argument as a lower case string. Otherwise return the value of the second argument as a string.

In [41]:
myFunc:{$[(abs type x)~11h;lower;::] //using abs to check for both atoms and lists
                                        //we return the function we want to use - lower, or null function
                                        //note no semicolon - the output of this will be applied to the LHS
   $[10h=type y;y;string y]   //return y as a string - leave alone if already a string
  } 

In [42]:
//testing
myFunc["thie";`BIG ]    //not a symbol with not a string - stays upper
myFunc[`this;`BIG  ]    //a symbol, and not a string
myFunc[`this;"BIG" ]    //a symbol and a string

BIG
big
big


In [45]:
// write your code here
function:{[arg1;arg2]
            $[(abs type arg1) = 11h; -1 string lower arg2;arg2]}

function["ARCHER";`ARCHWE]

ARCHWE


# Vector Conditional Evaluation
The vector conditional works with vectors and will evaluate the true or false expression for each item in the vector. Similar to `$`, (and unlike `if`) it provides a return value, but unlike the atomic equivalent `$`, this **cannot be extended to else-if clauses**. 

Vector condition evaluation has the form of :

Syntax:``?[x;y;z]`` where 

* `x` is a boolean vector
* `y` and `z` return the same type

In this case it will return the y values where x is true and z where x is false - a simple example: 

In [46]:
?[10001b;
    1 2 3 4 5;       //if true
    10 20 30 40 50]  //if false

1 20 30 40 5


In [47]:
?[10001b;1;10 20 30 40 50]          //if one of our conditions is atomic it will be repeated where necessary

1 20 30 40 1


In [49]:
//?[10001b;1;10 20 40 50]             //if providing a list to use, they need to be the same length

The below example will "cap" a vector at a maximum value, as provided: 

In [50]:
capAtMax:{[vector;cap]  
            ?[vector>cap; 
                cap;    //use the cap value where the vector value is bigger 
                vector] //use the original vector where it doesn't exceed the cap value
    }
show vec:30?20
capAtMax[vec;12]

9 8 1 9 12 12 12 1 12 6 1 12 6 5 12 12 8 0 12 5 12 2 0 0 8 7 12 12 10 12
9 8 1 9 17 14 13 1 18 6 1 14 6 5 17 17 8 0 16 5 13 2 0 0 8 7 18 13 10 15


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Vector Conditionals are what we use in qSQL queries, since columns in a table are really just lists! </i></p>

Given a table of trades we typically have a column side indicating whether this was a trade where we bought (Buy) or sold (Sell). We might also need the appropriate quote from the market. Normally we have two quotes- bid and ask. 

In [ ]:
side: `Buy`Buy`Sell
bids: 10.0 10.2 12.2
asks: 10.4 10.5 12.4  

To get the appropriate quote for our market trade we can use the following conditional: 

In [ ]:
?[side=`Buy;asks;bids]

##### Exercise 

Create a list `L` to be 1 2 3 0N 5. Using vector conditional evaluation, fill the null value with 10.

In [ ]:
L:1 2 3 0N 5;

?[null L;   //vector boolean check 
    10;     //if true return this
    L]      //if not true return this

In [52]:
//write your code here
L:1 2 3 0N 5
?[L=0N;10;L]

1 2 3 10 5


# Protected Evaluation 

As we have seen numerous times in the training, when kdb+/q encounters an error within code execution, the execution is halted i.e. no further statements are executed. In some cases we might want to just log an error and continue execution rather than completely halt our script or function - this is where we would use protected evaluation.

## Atomic Functions - Trap At @

We can use the [trap at](https://code.kx.com/q/ref/apply/#trap-at) operator which provides a way to capture errors raised during the application of the arguments to a function. The choice of what to do with the error is left to the user since it is captured as an argument to an error function. This function definition is a third argument to the operator.

Syntax:`@[function;arguments;errorFunction]` where the `errorFunction` is a function that takes only one argument - a string of the error raised during execution.

Perhaps best illustrated by an example: 

In [53]:
@[sin;`symbol;{[error] -2 "Error signal received:",error;}] 

Error signal received:type


The value returned from the failed function application is determined by the error function: 

In [54]:
@[sin;`symbol;{[error] "Just keep swimming!"}]  // we don't even use the error here!

Just keep swimming!


The error function is not special - it's just a standard function and we can use any function we like, provided it only requires one input.

This means we can use projections of multi-parameter functions, and this is commonly the case when we want to capture more detail about our error. 

In [55]:
argument:`symbol
handleError:{[arg;error]     2 "This broke it:",string[arg];
                            -2 "... type :",string type arg;
                            -2 "With error:",error;
                             0b};

errorFunction:handleError[argument]     //projecting to one argument

@[sin;argument;errorFunction]  

0b


This broke it:symbol... type :-11
With error:type


This can be called from within a function too: 

In [57]:
protectedSin:{[arg] @[sin;arg;handleError[arg]]}   //use protected evaluation to call sin with an argument

In [58]:
protectedSin[90]

0.8939967


In [59]:
protectedSin[`break]     //failure return 0b

0b


This broke it:break... type :-11
With error:type


In most cases where protected evaluation is used the error function defined will write details of the breakdown to a log file associated with the running process.

## Multivalent Functions - Trap . 

The operator to apply error trapping to a multivalent function is `.` and in its usage is syntactically the same as we saw with apply. The only difference is that the function is multivalent and therefore that there are multiple arguments, passed as a list. 

In [60]:
.[+; (1;`a);{-2"Error:",x;`Failed}]

Failed


Error:type


In [61]:
.[+; (1;2);{-2"Error:",x;`Failed}]

3


# Other useful (to know) control statements 

There are two other operators that we didn't mention above since they aren't used that often and if we don't have to use them, we **never** do. In saying this however, it is useful to know that they are there if we need to use them.

## Do
The [`do`](https://code.kx.com/q/ref/do/) operator allows us to repeat an execution *N* times. There is no return value from using `do`, similar to `if`.

Syntax: `do[NoOfTimes;expression;...;expressionN]`

The do statement can be used to time our q code:

In [ ]:
\t do[10;a*a:til 100000]
\t:10 a*a:til 100000     //Same as above - an example of the more "q" way!

## While

The [`while`](https://code.kx.com/q/ref/while/) operator allows the expression to be evaluated while the condition is true. Similar to `do`, `while` also does not return a value. 

Syntax:```while[condition;expression;...;expressionN]```

In [62]:
r:1 1
x:10
while[x-:1;r,:sum -2#r]  
r   //you may recognise this as the Fibonacci sequence! 

1 1 2 3 5 8 13 21 34 55 89
